In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.datasets import ImageFolder

In [2]:
class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super(SafeImageFolder, self).__getitem__(index)
        except FileNotFoundError:
            # If file is not found, return a dummy image and a dummy label
            dummy_img = Image.new('RGB', (128, 128))
            return self.transform(dummy_img), -1  # Use a label that indicates missing file


In [3]:

# Define the transformations for the training and validation datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ]),
}

# Specify the dataset directory paths
train_dir = r'C:\Users\User\Desktop\Tomato_disease\Datasets\tomato\train'
val_dir = r'C:\Users\User\Desktop\Tomato_disease\Datasets\tomato\val'

# Load the datasets using the SafeImageFolder class
train_dataset = SafeImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = SafeImageFolder(val_dir, transform=data_transforms['val'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [4]:
# Define CNN architecture 
class ImprovedPlantVillageCNN(nn.Module):
    def __init__(self):
        super(ImprovedPlantVillageCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(256 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model, criterion, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedPlantVillageCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training the model
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    running_correct = 0
    running_total = 0
    model.train()
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            if (labels == -1).all():  # Skip batches with only dummy data
                continue
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            running_total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
            pbar.set_postfix({'loss': running_loss / (pbar.n + 1)})
            pbar.update(1)

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(running_correct / running_total)

    # Validate the model every epoch
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        with tqdm(total=len(val_loader), desc='Validating', unit='batch') as pbar_val:
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                if (labels == -1).all():  # Skip batches with only dummy data
                    continue
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                pbar_val.update(1)

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(val_correct / val_total)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, Val Accuracy: {val_accuracies[-1]:.4f}')





Validating: 100%|██████████| 16/16 [00:05<00:00,  3.03batch/s]


Epoch [1/100], Train Loss: 1.5314, Val Loss: 1.4377, Train Accuracy: 0.5765, Val Accuracy: 0.6140


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.75batch/s]


Epoch [2/100], Train Loss: 0.6844, Val Loss: 1.3545, Train Accuracy: 0.7656, Val Accuracy: 0.6170


Validating: 100%|██████████| 16/16 [00:01<00:00, 13.63batch/s]


...

Epoch [98/100], Train Loss: 0.0294, Val Loss: 0.2284, Train Accuracy: 0.9903, Val Accuracy: 0.9570


Validating: 100%|██████████| 16/16 [00:01<00:00,  9.20batch/s]


Epoch [99/100], Train Loss: 0.0225, Val Loss: 0.7049, Train Accuracy: 0.9925, Val Accuracy: 0.9060


Validating: 100%|██████████| 16/16 [00:02<00:00,  7.27batch/s]

Epoch [100/100], Train Loss: 0.0337, Val Loss: 0.2426, Train Accuracy: 0.9899, Val Accuracy: 0.9560


In [5]:

# Save the trained model
model_save_path = 'model.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Model saved to model.pth
